In [6]:
# Instalar nuevos paquetes solo la primera vez
#!pip install geemap

In [7]:
# Llamamos a la libreria de gee API y paquetes
import ee
import datetime
import geemap

In [8]:
# Autorizar acceso a Google Earth Engine por unica vez
#ee.Authenticate()

# Iniciar Google Earth Engine cada vez que deseemos correr los codigos
ee.Initialize()

In [9]:
################################################## Data #######################################################################
# ID asset gee
territorio_nacional = ee.FeatureCollection('users/proyectosequiateleamb/Regiones/DPA_regiones_nacional')

# Colecciones de imagenes MODIS
MODIScollection = (ee.ImageCollection('MODIS/006/MOD10A1'))

# Se ordena la coleccion para que muestre la ultima imagen
MODISsorted = MODIScollection.sort('system:time_start', False).first()

# Muestra la fecha actual
currentDate = ee.Date(MODISsorted.get('system:time_start'))

# Muestra el mes anterior
Month = currentDate.getRelative('month', 'year');

# Muestra el año actual
YearCurrent = currentDate.get('year');

info1 = currentDate.getInfo()
print(info1)

info2 = Month.getInfo()
print(info2)

info3 = YearCurrent.getInfo()
print(info3)

{'type': 'Date', 'value': 1668988800000}
10
2022


In [14]:
################################################## Function #######################################################################
# Se define umbral NDSI
umbralNDSI = 40

# Se define una funcion para calcular SCI y CCI // En las funciones se debe tener consideracion con sagria en python
def snowcloudMask(image): 
    snow = image.select('NDSI_Snow_Cover').gte(umbralNDSI).multiply(100).rename('SCI')
    cloud = image.select('Snow_Albedo_Daily_Tile_Class').eq(150).multiply(100).rename('CCI')
    
    return image.addBands(snow).addBands(cloud)

# Coleccion con banda de nieve binario y nubes
Snow_cloud_collection = MODIScollection.map(snowcloudMask).select('SCI', 'CCI');

#print(Snow_cloud_collection)

info4 = Snow_cloud_collection.first().getInfo()
print(info4)

{'type': 'Image', 'bands': [{'id': 'SCI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 100}, 'crs': 'SR-ORG:6974', 'crs_transform': [463.3127165279165, 0, -20015109.353988, 0, -463.3127165274999, 10007554.676994]}, {'id': 'CCI', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 100}, 'crs': 'SR-ORG:6974', 'crs_transform': [463.3127165279165, 0, -20015109.353988, 0, -463.3127165274999, 10007554.676994]}], 'version': 1641324219874357, 'id': 'MODIS/006/MOD10A1/2000_02_24', 'properties': {'system:time_start': 951350400000, 'google:max_source_file_timestamp': 1513706761000, 'system:footprint': {'type': 'LinearRing', 'coordinates': [[-180, -90], [180, -90], [180, 90], [-180, 90], [-180, -90]]}, 'system:time_end': 951436800000, 'system:asset_size': 396169135, 'system:index': '2000_02_24'}}


In [11]:
############################################# Composition year and monthly #####################################################
# Lista de sequencia anual y mensual
years = ee.List.sequence(YearCurrent, YearCurrent);
months = ee.List.sequence(Month, Month);

info5 = years.getInfo()
print(info5)

info6 = months.getInfo()
print(info6)

[2022]
[10]


In [15]:
# Asigne una funcion para seleccionar datos dentro del año y aplique el reductor por promedio 
# Como resultado generara una nueva coleccion del promedio NDSI binario año y mes
def function1(y):
    def function2(m):
        return Snow_cloud_collection.filter(ee.Filter.calendarRange(y, y, 'year')).filter(ee.Filter.calendarRange(m, m, 'month')).mean().set('month', m).set('year', y).set('system:time_start', ee.Date.fromYMD(y, m, 1).format('YYYY-MM'));
    return months.map(function2)

MonthYearcollection = ee.ImageCollection.fromImages(years.map(function1).flatten())

In [16]:
print(MonthYearcollection.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'SCI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 100}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'CCI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 100}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': '2022-10', 'month': 10, 'year': 2022, 'system:index': '0'}}]}


In [17]:
# Muestra la ultima imagen con el promedio de sci y cci
last_image = MonthYearcollection.first()
print(last_image.getInfo())

{'type': 'Image', 'bands': [{'id': 'SCI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 100}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'CCI', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 100}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': '2022-10', 'month': 10, 'year': 2022, 'system:index': '0'}}


In [21]:
# Exportar ultima imagen de la coleccion a carpeta de Google Drive
property = last_image.get('system:time_start').getInfo()

task_config = {
        'image':last_image,
        'description': 'MOD10A1_SCI_CCI_' + property,
        'scale': 500,
        'region':territorio_nacional.geometry(),
        'maxPixels': 1E8,
        'folder': 'SCI&CCI'
    }
    
task = ee.batch.Export.image.toDrive(**task_config)
task.start()

In [23]:
# Se llama a la funcion map desde la libreria geemap
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [24]:
# Parametros de visualizacion del mapa
colorizedVis_SCI = {
  'min': 0.0,
  'max': 100.0,
  'palette': ['black', '0dffff', '0524ff', 'ffffff']
}

colorizedVis_CCI = {
  'min': 0.0,
  'max': 100.0,
  'palette': ['ffffff', '0dffff', '0524ff', 'gray'] 
}

Map.addLayer(last_image.clip(territorio_nacional).select('SCI'), colorizedVis_SCI, 'SCI')
Map.addLayer(last_image.clip(territorio_nacional).select('CCI'), colorizedVis_CCI, 'CCI')

Map.centerObject(territorio_nacional, 5)